In [2]:
!pip install fvcore

  Using cached fvcore-0.1.5.post20221221-py3-none-any.whl
  Using cached yacs-0.1.8-py3-none-any.whl.metadata (639 bytes)
  Using cached tabulate-0.9.0-py3-none-any.whl.metadata (34 kB)
  Using cached iopath-0.1.10-py3-none-any.whl
  Using cached portalocker-3.1.1-py3-none-any.whl.metadata (8.6 kB)
Using cached yacs-0.1.8-py3-none-any.whl (14 kB)
Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)
Using cached portalocker-3.1.1-py3-none-any.whl (19 kB)


In [6]:
checkpoint = torch.load("checkpoints/aebad_S_AeBAD_S_MMR_model.pth", map_location="cpu")
print(checkpoint.keys())


/tmp/ipykernel_1122262/1670864185.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("checkpoints/aebad_S_AeBAD_S_MMR_model.pth", map_location="cpu"

dict_keys(['cur_model_state_dict', 'mmr_base_state_dict', 'optimizer_state_dict', 'cfg'])


In [1]:
# Import required libraries
import torch
import torch.nn as nn
from torchvision import transforms, models
from PIL import Image
import numpy as np
from scipy.ndimage import gaussian_filter
from functools import partial
import matplotlib.pyplot as plt

# Import your model classes - adjust these imports based on your project structure
import sys
sys.path.append('./')  # Add project root to path
from models.MMR.MMR import MMR_base
from models.MMR.utils import ForwardHook, cal_anomaly_map
from config import get_cfg

# Initialize the anomaly detector
class AnomalyDetector:
    def __init__(self, checkpoint_path, config_path):
        # Load configuration
        self.cfg = get_cfg()
        self.cfg.merge_from_file(config_path)
        
        # Set device
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Using device: {self.device}")
        
        # Initialize models
        self.cur_model = self._init_current_model()
        self.mmr_model = self._init_mmr_model(checkpoint_path)
        
        # Register hooks
        self.teacher_outputs_dict = {}
        for extract_layer in self.cfg.TRAIN.MMR.layers_to_extract_from:
            forward_hook = ForwardHook(self.teacher_outputs_dict, extract_layer)
            network_layer = self.cur_model.__dict__["_modules"][extract_layer]
            network_layer[-1].register_forward_hook(forward_hook)
        
        # Define image transforms
        self.transform = transforms.Compose([
            transforms.Resize(self.cfg.DATASET.resize),
            transforms.CenterCrop(self.cfg.DATASET.imagesize),
            transforms.ToTensor(),
        ])

    def _init_current_model(self):
        try:
            print("Initializing WideResNet50 model...")
            model = models.wide_resnet50_2(pretrained=True)
            model.to(self.device)
            model.eval()
            print("WideResNet50 model initialized successfully")
            return model
        except Exception as e:
            raise Exception(f"Failed to initialize current model: {str(e)}")

    def _init_mmr_model(self, checkpoint_path: str):
        try:
            # Initialize the MMR base model
            model = MMR_base(cfg=self.cfg)

            # Load the checkpoint
            checkpoint = torch.load(checkpoint_path, map_location=self.device)

            # Load the correct state dictionary
            model.load_state_dict(checkpoint['mmr_base_state_dict'])

            # Send the model to the appropriate device and set to evaluation mode
            model.to(self.device)
            model.eval()
            print("MMR model initialized successfully")
            return model
        except Exception as e:
            print(f"Error details: {str(e)}")
            raise Exception(f"Failed to initialize MMR model: {str(e)}")


    def detect(self, image_path, threshold=0.4):
        """
        Detect anomalies in a single image
        Args:
            image_path: Path to the image file
            threshold: Anomaly threshold (0-1)
        Returns:
            anomaly_score: Overall anomaly score
            anomaly_map: 2D numpy array of anomaly scores
            is_anomaly: Boolean indicating if anomaly was detected
        """
        try:
            # Load and preprocess image
            image = Image.open(image_path).convert('RGB')
            image_tensor = self.transform(image).unsqueeze(0).to(self.device)
            
            with torch.no_grad():
                # Get current model features
                self.teacher_outputs_dict.clear()
                _ = self.cur_model(image_tensor)
                multi_scale_features = [self.teacher_outputs_dict[key] 
                                      for key in self.cfg.TRAIN.MMR.layers_to_extract_from]
                
                # Get MMR features
                reverse_features = self.mmr_model(image_tensor, 
                                               mask_ratio=self.cfg.TRAIN.MMR.test_mask_ratio)
                multi_scale_reverse_features = [reverse_features[key] 
                                              for key in self.cfg.TRAIN.MMR.layers_to_extract_from]
                
                # Calculate anomaly map
                anomaly_map, _ = cal_anomaly_map(
                    multi_scale_features, 
                    multi_scale_reverse_features, 
                    image_tensor.shape[-1],
                    amap_mode='a'
                )
                
                # Apply Gaussian smoothing
                anomaly_map = gaussian_filter(anomaly_map[0], sigma=4)
                
                # Calculate anomaly score
                anomaly_score = np.max(anomaly_map)
                is_anomaly = anomaly_score > threshold
                
                return {
                    'anomaly_score': float(anomaly_score),
                    'anomaly_map': anomaly_map,
                    'is_anomaly': bool(is_anomaly),
                    'original_image': image
                }
        except Exception as e:
            raise Exception(f"Error during detection: {str(e)}")

    def visualize_result(self, result):
        """
        Visualize the detection result
        Args:
            result: Dictionary containing detection results
        """
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
        
        # Plot original image
        ax1.imshow(result['original_image'])
        ax1.set_title('Original Image')
        ax1.axis('off')
        
        # Plot anomaly map
        im = ax2.imshow(result['anomaly_map'], cmap='jet')
        ax2.set_title(f"Anomaly Map\nScore: {result['anomaly_score']:.3f}")
        ax2.axis('off')
        
        # Add colorbar
        plt.colorbar(im, ax=ax2)
        plt.tight_layout()
        plt.show()

# Create detector instance
detector = AnomalyDetector(
    checkpoint_path="checkpoints/aebad_S_AeBAD_S_MMR_model.pth",
    config_path="method_config/AeBAD_S/MMR.yaml"
)



Using device: cuda
Initializing WideResNet50 model...


/home/rvce/durgarao/apps/miniconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rvce/durgarao/apps/miniconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Wide_ResNet50_2_Weights.IMAGENET1K_V1`. You can also use `weights=Wide_ResNet50_2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


WideResNet50 model initialized successfully


/tmp/ipykernel_1126766/581795970.py:64: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=self.device)


MMR model initialized successfully


In [3]:
import glob
import os
from collections import defaultdict

# Define directories for test images
base_directory = "/home/rvce/Downloads/aircraft/AeBAD/AeBAD_S/test/"
categories = {
    "good": "good",  # Good images
    "ablation": "ablation",  # Anomaly: Ablation
    "breakdown": "breakdown",  # Anomaly: Breakdown
    "fracture": "fracture",  # Anomaly: Fracture
    "groove": "groove",  # Anomaly: Groove
}

conditions = ["background", "illumination", "same", "view"]

# Initialize results storage
overall_results = defaultdict(list)

# Function to process and detect anomalies in images from a specific directory
def process_directory(detector, category, condition):
    print(f"\nProcessing {category.capitalize()} images under '{condition}' condition...")
    image_directory = os.path.join(base_directory, category, condition)
    image_paths = glob.glob(os.path.join(image_directory, "*.png"))
    
    if not image_paths:
        print(f"No images found in {image_directory}")
        return
    
    print(f"Found {len(image_paths)} images.")
    
    for image_path in image_paths:
        print(f"Processing image: {os.path.basename(image_path)}")
        try:
            # Detect anomaly for the current image
            result = detector.detect(image_path)
            
            # Print results for the image
            print(f"  Anomaly Score: {result['anomaly_score']:.3f}")
            print(f"  Is Anomaly: {result['is_anomaly']}")
            
            # Store the result
            overall_results[category].append({
                "image_path": image_path,
                "anomaly_score": result['anomaly_score'],
                "is_anomaly": result['is_anomaly'],
            })
            
            # Visualize the result
            detector.visualize_result(result)
        except Exception as e:
            print(f"Error processing image {image_path}: {str(e)}")

# Loop through all categories and conditions
for category, folder in categories.items():
    for condition in conditions:
        process_directory(detector, folder, condition)

# Print overall summary
print("\n=== Overall Summary ===")
for category, results in overall_results.items():
    print(f"\nCategory: {category.capitalize()}")
    for res in results:
        print(f"  Image: {os.path.basename(res['image_path'])}, "
              f"Anomaly Score: {res['anomaly_score']:.3f}, Is Anomaly: {res['is_anomaly']}")



Processing Good images under 'background' condition...
Found 93 images.
Processing image: IMG_7766.png
  Anomaly Score: 0.406
  Is Anomaly: True
Processing image: IMG_7795.png
  Anomaly Score: 0.401
  Is Anomaly: True
Processing image: IMG_7738.png
  Anomaly Score: 0.383
  Is Anomaly: False
Processing image: IMG_7740.png
  Anomaly Score: 0.460
  Is Anomaly: True
Processing image: IMG_7758.png
  Anomaly Score: 0.360
  Is Anomaly: False
Processing image: IMG_7787.png
  Anomaly Score: 0.382
  Is Anomaly: False
Processing image: IMG_8129.png
  Anomaly Score: 0.351
  Is Anomaly: False
Processing image: IMG_7776.png
  Anomaly Score: 0.364
  Is Anomaly: False
Processing image: IMG_8145.png
  Anomaly Score: 0.354
  Is Anomaly: False
Processing image: IMG_7737.png
  Anomaly Score: 0.382
  Is Anomaly: False
Processing image: IMG_7779.png
  Anomaly Score: 0.384
  Is Anomaly: False
Processing image: IMG_7765.png
  Anomaly Score: 0.336
  Is Anomaly: False
Processing image: IMG_8120.png
  Anomaly S

Processing image: IMG_7974.png
  Anomaly Score: 0.364
  Is Anomaly: False
Processing image: IMG_7709.png
  Anomaly Score: 0.411
  Is Anomaly: True
Processing image: IMG_7720.png
  Anomaly Score: 0.348
  Is Anomaly: False
Processing image: IMG_7712.png
  Anomaly Score: 0.354
  Is Anomaly: False
Processing image: IMG_7704.png
  Anomaly Score: 0.371
  Is Anomaly: False
Processing image: IMG_7710.png
  Anomaly Score: 0.459
  Is Anomaly: True
Processing image: IMG_7984.png
  Anomaly Score: 0.384
  Is Anomaly: False
Processing image: IMG_8182.png
  Anomaly Score: 0.426
  Is Anomaly: True
Processing image: IMG_7972.png
  Anomaly Score: 0.452
  Is Anomaly: True
Processing image: IMG_7701.png
  Anomaly Score: 0.328
  Is Anomaly: False
Processing image: IMG_7952.png
  Anomaly Score: 0.380
  Is Anomaly: False
Processing image: IMG_7718.png
  Anomaly Score: 0.335
  Is Anomaly: False
Processing image: IMG_8192.png
  Anomaly Score: 0.342
  Is Anomaly: False
Processing image: IMG_8196.png
  Anomaly S

Processing image: IMG_7583.png
  Anomaly Score: 0.240
  Is Anomaly: False
Processing image: IMG_7811.png
  Anomaly Score: 0.407
  Is Anomaly: True
Processing image: IMG_7820.png
  Anomaly Score: 0.418
  Is Anomaly: True
Processing image: IMG_7884.png
  Anomaly Score: 0.365
  Is Anomaly: False
Processing image: IMG_7672.png
  Anomaly Score: 0.372
  Is Anomaly: False
Processing image: IMG_7888.png
  Anomaly Score: 0.380
  Is Anomaly: False
Processing image: IMG_7815.png
  Anomaly Score: 0.329
  Is Anomaly: False
Processing image: IMG_8010.png
  Anomaly Score: 0.346
  Is Anomaly: False
Processing image: IMG_7994.png
  Anomaly Score: 0.354
  Is Anomaly: False
Processing image: IMG_8002.png
  Anomaly Score: 0.396
  Is Anomaly: False
Processing image: IMG_8045.png
  Anomaly Score: 0.383
  Is Anomaly: False
Processing image: IMG_7862.png
  Anomaly Score: 0.466
  Is Anomaly: True
Processing image: IMG_7995.png
  Anomaly Score: 0.444
  Is Anomaly: True
Processing image: IMG_7673.png
  Anomaly S

Processing image: IMG_8021.png
  Anomaly Score: 0.361
  Is Anomaly: False
Processing image: IMG_8049.png
  Anomaly Score: 0.338
  Is Anomaly: False
Processing image: IMG_7623.png
  Anomaly Score: 0.359
  Is Anomaly: False
Processing image: IMG_7670.png
  Anomaly Score: 0.410
  Is Anomaly: True
Processing image: IMG_7816.png
  Anomaly Score: 0.328
  Is Anomaly: False
Processing image: IMG_7838.png
  Anomaly Score: 0.366
  Is Anomaly: False
Processing image: IMG_7661.png
  Anomaly Score: 0.394
  Is Anomaly: False
Processing image: IMG_7990.png
  Anomaly Score: 0.351
  Is Anomaly: False
Processing image: IMG_8032.png
  Anomaly Score: 0.299
  Is Anomaly: False
Processing image: IMG_8047.png
  Anomaly Score: 0.441
  Is Anomaly: True
Processing image: IMG_7860.png
  Anomaly Score: 0.420
  Is Anomaly: True
Processing image: IMG_7908.png
  Anomaly Score: 0.463
  Is Anomaly: True
Processing image: IMG_7819.png
  Anomaly Score: 0.284
  Is Anomaly: False
Processing image: IMG_7685.png
  Anomaly S

Processing image: IMG_8084.png
  Anomaly Score: 0.305
  Is Anomaly: False
Processing image: IMG_8103.png
  Anomaly Score: 0.430
  Is Anomaly: True
Processing image: IMG_7652.png
  Anomaly Score: 0.397
  Is Anomaly: False
Processing image: IMG_7937.png
  Anomaly Score: 0.386
  Is Anomaly: False
Processing image: IMG_7949.png
  Anomaly Score: 0.458
  Is Anomaly: True
Processing image: IMG_8112.png
  Anomaly Score: 0.411
  Is Anomaly: True
Processing image: IMG_7931.png
  Anomaly Score: 0.391
  Is Anomaly: False
Processing image: IMG_7915.png
  Anomaly Score: 0.448
  Is Anomaly: True
Processing image: IMG_8107.png
  Anomaly Score: 0.411
  Is Anomaly: True
Processing image: IMG_8067.png
  Anomaly Score: 0.406
  Is Anomaly: True
Processing image: IMG_7639.png
  Anomaly Score: 0.402
  Is Anomaly: True
Processing image: IMG_8115.png
  Anomaly Score: 0.431
  Is Anomaly: True
Processing image: IMG_7919.png
  Anomaly Score: 0.373
  Is Anomaly: False
Processing image: IMG_7640.png
  Anomaly Score

...

Processing image: IMG_9741.png
  Anomaly Score: 0.503
  Is Anomaly: True
Processing image: IMG_9727.png
  Anomaly Score: 0.478
  Is Anomaly: True
Processing image: IMG_9699.png
  Anomaly Score: 0.497
  Is Anomaly: True
Processing image: IMG_9753.png
  Anomaly Score: 0.474
  Is Anomaly: True
Processing image: IMG_9720.png
  Anomaly Score: 0.467
  Is Anomaly: True

Processing Fracture images under 'same' condition...
Found 161 images.
Processing image: IMG_9466.png
  Anomaly Score: 0.404
  Is Anomaly: True
Processing image: IMG_9547.png
  Anomaly Score: 0.404
  Is Anomaly: True
Processing image: IMG_9520.png
  Anomaly Score: 0.413
  Is Anomaly: True
Processing image: IMG_9448.png
  Anomaly Score: 0.371
  Is Anomaly: False
Processing image: IMG_9553.png
  Anomaly Score: 0.443
  Is Anomaly: True
Processing image: IMG_9544.png
  Anomaly Score: 0.422
  Is Anomaly: True
Processing image: IMG_9436.png
  Anomaly Score: 0.544
  Is Anomaly: True
Processing image: IMG_9481.png
  Anomaly Score: 0.4

Processing image: IMG_9482.png
  Anomaly Score: 0.419
  Is Anomaly: True
Processing image: IMG_9443.png
  Anomaly Score: 0.521
  Is Anomaly: True
Processing image: IMG_9439.png
  Anomaly Score: 0.350
  Is Anomaly: False
Processing image: IMG_9490.png
  Anomaly Score: 0.405
  Is Anomaly: True
Processing image: IMG_9406.png
  Anomaly Score: 0.475
  Is Anomaly: True
Processing image: IMG_9434.png
  Anomaly Score: 0.495
  Is Anomaly: True
Processing image: IMG_9573.png
  Anomaly Score: 0.434
  Is Anomaly: True
Processing image: IMG_9561.png
  Anomaly Score: 0.483
  Is Anomaly: True
Processing image: IMG_9468.png
  Anomaly Score: 0.450
  Is Anomaly: True
Processing image: IMG_9550.png
  Anomaly Score: 0.370
  Is Anomaly: False
Processing image: IMG_9418.png
  Anomaly Score: 0.464
  Is Anomaly: True
Processing image: IMG_9509.png
  Anomaly Score: 0.404
  Is Anomaly: True
Processing image: IMG_9453.png
  Anomaly Score: 0.497
  Is Anomaly: True
Processing image: IMG_9510.png
  Anomaly Score: 0

Processing image: IMG_9685.png
  Anomaly Score: 0.421
  Is Anomaly: True
Processing image: IMG_9603.png
  Anomaly Score: 0.423
  Is Anomaly: True
Processing image: IMG_9643.png
  Anomaly Score: 0.505
  Is Anomaly: True
Processing image: IMG_9587.png
  Anomaly Score: 0.380
  Is Anomaly: False
Processing image: IMG_9659.png
  Anomaly Score: 0.489
  Is Anomaly: True
Processing image: IMG_9673.png
  Anomaly Score: 0.432
  Is Anomaly: True
Processing image: IMG_9667.png
  Anomaly Score: 0.491
  Is Anomaly: True
Processing image: IMG_9687.png
  Anomaly Score: 0.467
  Is Anomaly: True
Processing image: IMG_9683.png
  Anomaly Score: 0.474
  Is Anomaly: True
Processing image: IMG_9648.png
  Anomaly Score: 0.400
  Is Anomaly: False
Processing image: IMG_9604.png
  Anomaly Score: 0.514
  Is Anomaly: True
Processing image: IMG_9628.png
  Anomaly Score: 0.473
  Is Anomaly: True
Processing image: IMG_9680.png
  Anomaly Score: 0.430
  Is Anomaly: True
Processing image: IMG_9688.png
  Anomaly Score: 0

Processing image: IMG_8524.png
  Anomaly Score: 0.499
  Is Anomaly: True
Processing image: IMG_8531.png
  Anomaly Score: 0.461
  Is Anomaly: True
Processing image: IMG_8507.png
  Anomaly Score: 0.452
  Is Anomaly: True
Processing image: IMG_8526.png
  Anomaly Score: 0.420
  Is Anomaly: True
Processing image: IMG_8457.png
  Anomaly Score: 0.395
  Is Anomaly: False
Processing image: IMG_8494.png
  Anomaly Score: 0.402
  Is Anomaly: True
Processing image: IMG_8500.png
  Anomaly Score: 0.404
  Is Anomaly: True
Processing image: IMG_8530.png
  Anomaly Score: 0.392
  Is Anomaly: False
Processing image: IMG_8522.png
  Anomaly Score: 0.493
  Is Anomaly: True
Processing image: IMG_8521.png
  Anomaly Score: 0.405
  Is Anomaly: True
Processing image: IMG_8515.png
  Anomaly Score: 0.409
  Is Anomaly: True
Processing image: IMG_8517.png
  Anomaly Score: 0.405
  Is Anomaly: True
Processing image: IMG_8511.png
  Anomaly Score: 0.395
  Is Anomaly: False
Processing image: IMG_8485.png
  Anomaly Score: 

Processing image: IMG_8214.png
  Anomaly Score: 0.446
  Is Anomaly: True
Processing image: IMG_8215.png
  Anomaly Score: 0.439
  Is Anomaly: True
Processing image: IMG_8245.png
  Anomaly Score: 0.500
  Is Anomaly: True
Processing image: IMG_8283.png
  Anomaly Score: 0.390
  Is Anomaly: False
Processing image: IMG_8273.png
  Anomaly Score: 0.403
  Is Anomaly: True
Processing image: IMG_8257.png
  Anomaly Score: 0.388
  Is Anomaly: False
Processing image: IMG_8322.png
  Anomaly Score: 0.366
  Is Anomaly: False
Processing image: IMG_8296.png
  Anomaly Score: 0.357
  Is Anomaly: False
Processing image: IMG_8237.png
  Anomaly Score: 0.418
  Is Anomaly: True
Processing image: IMG_8293.png
  Anomaly Score: 0.355
  Is Anomaly: False
Processing image: IMG_8205.png
  Anomaly Score: 0.405
  Is Anomaly: True
Processing image: IMG_8244.png
  Anomaly Score: 0.419
  Is Anomaly: True
Processing image: IMG_8251.png
  Anomaly Score: 0.410
  Is Anomaly: True
Processing image: IMG_8243.png
  Anomaly Score

In [5]:
# Print overall summary with details
print("\n=== Overall Summary ===")
category_summary = defaultdict(lambda: defaultdict(lambda: {"anomalous": 0, "non_anomalous": 0}))

for category, results in overall_results.items():
    print(f"\nCategory: {category.capitalize()}")
    for res in results:
        image_name = os.path.basename(res['image_path'])
        anomaly_status = "Anomalous" if res['is_anomaly'] else "Non-Anomalous"
        print(f"  Image: {image_name}, Anomaly Score: {res['anomaly_score']:.3f}, Is Anomaly: {res['is_anomaly']}")

        # Update category and condition summary
        condition = os.path.basename(os.path.dirname(res['image_path']))  # Get condition (e.g., background, view)
        if res['is_anomaly']:
            category_summary[category][condition]["anomalous"] += 1
        else:
            category_summary[category][condition]["non_anomalous"] += 1

# Print detailed summary
print("\n=== Anomalous vs Non-Anomalous Summary ===")
for category, conditions in category_summary.items():
    print(f"\nCategory: {category.capitalize()}")
    for condition, counts in conditions.items():
        print(f"  Condition: {condition.capitalize()}")
        print(f"    Anomalous: {counts['anomalous']}")
        print(f"    Non-Anomalous: {counts['non_anomalous']}")



=== Overall Summary ===

Category: Good
  Image: IMG_7766.png, Anomaly Score: 0.406, Is Anomaly: True
  Image: IMG_7795.png, Anomaly Score: 0.401, Is Anomaly: True
  Image: IMG_7738.png, Anomaly Score: 0.383, Is Anomaly: False
  Image: IMG_7740.png, Anomaly Score: 0.460, Is Anomaly: True
  Image: IMG_7758.png, Anomaly Score: 0.360, Is Anomaly: False
  Image: IMG_7787.png, Anomaly Score: 0.382, Is Anomaly: False
  Image: IMG_8129.png, Anomaly Score: 0.351, Is Anomaly: False
  Image: IMG_7776.png, Anomaly Score: 0.364, Is Anomaly: False
  Image: IMG_8145.png, Anomaly Score: 0.354, Is Anomaly: False
  Image: IMG_7737.png, Anomaly Score: 0.382, Is Anomaly: False
  Image: IMG_7779.png, Anomaly Score: 0.384, Is Anomaly: False
  Image: IMG_7765.png, Anomaly Score: 0.336, Is Anomaly: False
  Image: IMG_8120.png, Anomaly Score: 0.382, Is Anomaly: False
  Image: IMG_7748.png, Anomaly Score: 0.327, Is Anomaly: False
  Image: IMG_8139.png, Anomaly Score: 0.360, Is Anomaly: False
  Image: IMG_7786